----
# Making plaquettes
----

The idea is having a mostly-not-visual tool in the form of a Python module with a function which receives:
- a list of data qubits
- a list of stabilizers
and creates an internal representation (IR) of the code assuming that an extra ancilla qubit is available with the right connectivity.

One can then emit the `stim` code from the IR of the code.

To illustrate the idea, with many suboptimal choices `:-)`, I added a couple of utility functions to `utils.py` and a small graphical interface to `gui.py`.  
Note that using a networkx graph as IR for the QEC code is suboptimal.  
For example, it requires additional info to identify data, x-ancilla, and z-ancilla qubits.

In this context, easy improvements are:
- function to prune the edges between data qubits
- function to save/load the IR (here a graph)
- function to emit STIM code assuming native connectivity
- function to check code compatibility (i.e. data qubit exists, stabilizers pairwise commute, ...)
- function to print the IR which does not rotate the graph and understands the overall 2d-grid structure

Sections:
- <a href=#board>Make the board</a>
- <a href=#stim>Emit stim code</a>

----
<a id='top'></a>

In [3]:
# Set working directory to be the main directory of the repo.
import os
print('working directory:', os.getcwd())

import numpy as np              # Import NumPy library
import pandas as pd
import networkx as nx

#from utils import custom_grid_2d_graph, is_stabilizer, add_stabilizer_to_graph, shift_stabilizer
from gui import launch_interactive_composer

working directory: /data/nfs_home/giangiac/tqec/proj_making-plaquettes


<a id='board'></a>

----
## Make the board

<a href=#top>Back to top</a>

In [4]:
launch_interactive_composer()


<a id='stim'></a>

----
## Convert to stim code

<a href=#top>Back to top</a>

----
## END
----